In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
!pip install fastai==0.7.0
from fastai.imports import *
from fastai.structured import *

from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display

from sklearn import metrics

    100% |████████████████████████████████| 122kB 5.9MB/s ta 0:00:01
    100% |████████████████████████████████| 496.4MB 63kB/s  eta 0:00:01   16% |█████▍                          | 83.5MB 53.6MB/s eta 0:00:08    19% |██████▎                         | 98.2MB 60.9MB/s eta 0:00:07    50% |████████████████                | 248.5MB 46.9MB/s eta 0:00:06��███████▉               | 261.6MB 73.0MB/s eta 0:00:04    68% |█████████████████████▉          | 339.4MB 34.9MB/s eta 0:00:05    71% |███████████████████████         | 356.4MB 43.6MB/s eta 0:00:04


  Found existing installation: torch 1.0.1.post2
    Uninstalling torch-1.0.1.post2:
      Successfully uninstalled torch-1.0.1.post2
  Found existing installation: fastai 1.0.51
    Uninstalling fastai-1.0.51:
      Successfully uninstalled fastai-1.0.51
You are using pip version 19.0.3, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
PATH = "../input/"
!ls {PATH}

Data Dictionary.xlsx  Valid.csv				train
Machine_Appendix.csv  ValidSolution.csv			trainandvalid
Test.csv	      median_benchmark.csv		valid
TrainAndValid.csv     random_forest_benchmark_test.csv


In [33]:
df = pd.read_csv(f'{PATH}train/Train.csv', low_memory=False, parse_dates=['saledate'])

In [34]:
def display_all(df):
    with pd.option_context("display.max_rows", 1000, "display.max_columns", 1000): 
        display(df)

In [35]:
df.shape

(401125, 53)

In [36]:
display_all(df.head().T)

,0,1,2,3,4
SalesID,1139246,1139248,1139249,1139251,1139253
SalePrice,66000,57000,10000,38500,11000
MachineID,999089,117657,434808,1026470,1057373
ModelID,3157,77,7009,332,17311
datasource,121,121,121,121,121
auctioneerID,3,3,3,3,3
YearMade,2004,1996,2001,2001,2007
MachineHoursCurrentMeter,68,4640,2838,3486,722
UsageBand,Low,Low,High,High,Medium
saledate,2006-11-16 00:00:00,2004-03-26 00:00:00,2004-02-26 00:00:00,2011-05-19 00:00:00,2009-07-23 00:00:00


In [37]:
display_all(df.describe(include='all').T)

,count,unique,top,freq,first,last,mean,std,min,25%,50%,75%,max
SalesID,401125,NaN,NaN,NaN,NaN,NaN,1.91971e+06,909021,1.13925e+06,1.41837e+06,1.63942e+06,2.24271e+06,6.33334e+06
SalePrice,401125,NaN,NaN,NaN,NaN,NaN,31099.7,23036.9,4750,14500,24000,40000,142000
MachineID,401125,NaN,NaN,NaN,NaN,NaN,1.2179e+06,440992,0,1.0887e+06,1.27949e+06,1.46807e+06,2.48633e+06
ModelID,401125,NaN,NaN,NaN,NaN,NaN,6889.7,6221.78,28,3259,4604,8724,37198
datasource,401125,NaN,NaN,NaN,NaN,NaN,134.666,8.96224,121,132,132,136,172
auctioneerID,380989,NaN,NaN,NaN,NaN,NaN,6.55604,16.9768,0,1,2,4,99
YearMade,401125,NaN,NaN,NaN,NaN,NaN,1899.16,291.797,1000,1985,1995,2000,2013
MachineHoursCurrentMeter,142765,NaN,NaN,NaN,NaN,NaN,3457.96,27590.3,0,0,0,3025,2.4833e+06
UsageBand,69639,3,Medium,33985,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
saledate,401125,3919,2009-02-16 00:00:00,1932,1989-01-17 00:00:00,2011-12-30 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
# df.SalePrice = np.log(df.SalePrice)

**Data processing**

*Add dateparts from saledate which contains lots of important and helpful data*

In [39]:
add_datepart(df, 'saledate')
df.columns

Index(['SalesID', 'SalePrice', 'MachineID', 'ModelID', 'datasource',
       'auctioneerID', 'YearMade', 'MachineHoursCurrentMeter', 'UsageBand',
       'fiModelDesc', 'fiBaseModel', 'fiSecondaryDesc', 'fiModelSeries',
       'fiModelDescriptor', 'ProductSize', 'fiProductClassDesc', 'state',
       'ProductGroup', 'ProductGroupDesc', 'Drive_System', 'Enclosure',
       'Forks', 'Pad_Type', 'Ride_Control', 'Stick', 'Transmission',
       'Turbocharged', 'Blade_Extension', 'Blade_Width', 'Enclosure_Type',
       'Engine_Horsepower', 'Hydraulics', 'Pushblock', 'Ripper', 'Scarifier',
       'Tip_Control', 'Tire_Size', 'Coupler', 'Coupler_System',
       'Grouser_Tracks', 'Hydraulics_Flow', 'Track_Type',
       'Undercarriage_Pad_Width', 'Stick_Length', 'Thumb', 'Pattern_Changer',
       'Grouser_Type', 'Backhoe_Mounting', 'Blade_Type', 'Travel_Controls',
       'Differential_Type', 'Steering_Controls', 'saleYear', 'saleMonth',
       'saleWeek', 'saleDay', 'saleDayofweek', 'saleDayofyear',


*Replace catergorical values with numeric value*

In [40]:
train_cats(df)
display_all(df.head().T)

,0,1,2,3,4
SalesID,1139246,1139248,1139249,1139251,1139253
SalePrice,66000,57000,10000,38500,11000
MachineID,999089,117657,434808,1026470,1057373
ModelID,3157,77,7009,332,17311
datasource,121,121,121,121,121
auctioneerID,3,3,3,3,3
YearMade,2004,1996,2001,2001,2007
MachineHoursCurrentMeter,68,4640,2838,3486,722
UsageBand,Low,Low,High,High,Medium
fiModelDesc,521D,950FII,226,PC120-6E,S175


In [41]:
df.UsageBand.cat.categories

Index(['High', 'Low', 'Medium'], dtype='object')

In [42]:
df.UsageBand.cat.set_categories(['High', 'Medium', 'Low'], ordered=True, inplace=True)

In [43]:
df.UsageBand = df.UsageBand.cat.codes

In [44]:
display_all(df.head().T)

,0,1,2,3,4
SalesID,1139246,1139248,1139249,1139251,1139253
SalePrice,66000,57000,10000,38500,11000
MachineID,999089,117657,434808,1026470,1057373
ModelID,3157,77,7009,332,17311
datasource,121,121,121,121,121
auctioneerID,3,3,3,3,3
YearMade,2004,1996,2001,2001,2007
MachineHoursCurrentMeter,68,4640,2838,3486,722
UsageBand,2,2,0,0,1
fiModelDesc,521D,950FII,226,PC120-6E,S175


*Handle missing values, and split data into dependent and independent variables*

In [45]:
display_all(df.isnull().sum().sort_index()/len(df))

Backhoe_Mounting            0.803872
Blade_Extension             0.937129
Blade_Type                  0.800977
Blade_Width                 0.937129
Coupler                     0.466620
Coupler_System              0.891660
Differential_Type           0.826959
Drive_System                0.739829
Enclosure                   0.000810
Enclosure_Type              0.937129
Engine_Horsepower           0.937129
Forks                       0.521154
Grouser_Tracks              0.891899
Grouser_Type                0.752813
Hydraulics                  0.200823
Hydraulics_Flow             0.891899
MachineHoursCurrentMeter    0.644089
MachineID                   0.000000
ModelID                     0.000000
Pad_Type                    0.802720
Pattern_Changer             0.752651
ProductGroup                0.000000
ProductGroupDesc            0.000000
ProductSize                 0.525460
Pushblock                   0.937129
Ride_Control                0.629527
Ripper                      0.740388
S

In [46]:
df, y, nas = proc_df(df, 'SalePrice')

In [47]:
os.makedirs('tmp', exist_ok=True)
df.to_feather('tmp/bulldozers-raw')

In [48]:
df_raw = pd.read_feather('tmp/bulldozers-raw')

/opt/conda/lib/python3.6/site-packages/pandas/io/feather_format.py:112: FutureWarning: `nthreads` argument is deprecated, pass `use_threads` instead
  return feather.read_dataframe(path, nthreads=nthreads)


**Train and test model**

In [49]:
rf = RandomForestRegressor(n_jobs=-1)
rf.fit(df, y)
rf.score(df, y)

/opt/conda/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.9827086332492472

*Split train-test data*

In [50]:
def split_val(df, n):
    return df[:n].copy(), df[n:].copy()

n_val = 12000
n_trn = len(df) - n_val
x_trn, x_val = split_val(df, n_trn)
y_trn, y_val = split_val(y, n_trn)
x_trn.shape, x_val.shape, y_trn.shape, y_val.shape

((389125, 66), (12000, 66), (389125,), (12000,))

**Basic Random Forest**

In [51]:
def rmse(x, y):
    return math.sqrt(((np.log(x)-np.log(y))**2).mean())

def print_score(m):
    res = [rmse(m.predict(x_trn), y_trn), rmse(m.predict(x_val), y_val),
                m.score(x_trn, y_trn), m.score(x_val, y_val)]
    print(res)

In [52]:
rf = RandomForestRegressor(n_jobs=-1)
%time rf.fit(x_trn, y_trn)
print_score(rf)

/opt/conda/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


CPU times: user 1min 45s, sys: 572 ms, total: 1min 45s
Wall time: 31.5 s
[0.09728232841046859, 0.26314490731885265, 0.9825597129413558, 0.8799408916375258]


**Predict on test data**

In [62]:
test_df = pd.read_csv(f'{PATH}Test.csv', low_memory=False, parse_dates=['saledate'])

In [54]:
add_datepart(test_df, 'saledate')

In [55]:
train_cats(test_df)

In [56]:
test_df.UsageBand.cat.set_categories(['High', 'Medium', 'Low'], ordered=True, inplace=True)
test_df.UsageBand = test_df.UsageBand.cat.codes

In [57]:
display_all(test_df.isnull().sum().sort_index()/len(test_df))

Backhoe_Mounting            0.835354
Blade_Extension             0.947740
Blade_Type                  0.834792
Blade_Width                 0.947740
Coupler                     0.389821
Coupler_System              0.834149
Differential_Type           0.829092
Drive_System                0.778518
Enclosure                   0.000161
Enclosure_Type              0.947740
Engine_Horsepower           0.947740
Forks                       0.493618
Grouser_Tracks              0.834149
Grouser_Type                0.727543
Hydraulics                  0.171952
Hydraulics_Flow             0.834149
MachineHoursCurrentMeter    0.829092
MachineID                   0.000000
ModelID                     0.000000
Pad_Type                    0.830778
Pattern_Changer             0.727543
ProductGroup                0.000000
ProductGroupDesc            0.000000
ProductSize                 0.514490
Pushblock                   0.947740
Ride_Control                0.659549
Ripper                      0.782933
S

In [58]:
test_df, _, _ = proc_df(test_df, na_dict=nas)

In [59]:
output_df = pd.DataFrame({
    'SalesId': test_df.SalesID,
    'SalePrice': rf.predict(test_df)
})

In [60]:
output_df.tail()

,SalesId,SalePrice
12452,6643171,62650.0
12453,6643173,12600.0
12454,6643184,13525.0
12455,6643186,17950.0
12456,6643196,26100.0


In [61]:
output_df.to_csv('bulldozer_test_result_00.csv')